In [32]:
from PIL import Image, ImageOps, ImageDraw, ImageFont
import numpy as np
from matplotlib import pyplot as plt

import torch

In [60]:
# im = ImageOps.grayscale(Image.open('sampleResult1.png'))
im = ImageOps.grayscale(Image.open('toy1.png'))
# im.show()

im_original = torch.Tensor(np.array(im).astype(np.float64))
print(im_original.max())

im_original = im_original.cuda()

tensor(255.)


In [61]:
# def concat_images_vertically(image_a, image_b, padding):
#     width = max(image_a.width, image_b.width)
#     height = image_a.height + padding + image_b.height

#     new_image = Image.new('RGB', (width, height), (255, 255, 255))  # Create a new blank image

#     new_image.paste(image_a, (0, 0))  # Paste image_a at the top left corner
#     new_image.paste(image_b, (0, image_a.height + padding))  # Paste image_b with padding

#     return new_image

def concat_images_vertically(image_a, image_b, padding, label_a=None, label_b=None):
    width = max(image_a.width, image_b.width)
    height = image_a.height + padding + image_b.height

    new_image = Image.new('RGB', (width, height), (255, 255, 255))  # Create a new blank image

    new_image.paste(image_a, (0, 0))  # Paste image_a at the top left corner
    new_image.paste(image_b, (0, image_a.height + padding))  # Paste image_b with padding

    draw = ImageDraw.Draw(new_image)
    label_font = ImageFont.truetype('arial.ttf', 16)  # Change the font and size as needed

    if label_a is not None:
        label_width, label_height = draw.textsize(label_a, font=label_font)
        label_position_a = (width // 2 - label_width // 2, label_height // 2)
        draw.text(label_position_a, label_a, font=label_font, fill=(0, 0, 0))

    if label_b is not None:
        label_width, label_height = draw.textsize(label_b, font=label_font)
        label_position_b = (width // 2 - label_width // 2, image_a.height + padding + label_height // 2)
        draw.text(label_position_b, label_b, font=label_font, fill=(0, 0, 0))

    return new_image

In [68]:
valid_choices = torch.Tensor([
    [0, 0, 0, 0],
#     [0.5, 0.5, 0.5, 0.5],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 1],
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
]).to(im_original.device)

print(im_original.shape)

W = im_original.shape[0]
H = im_original.shape[1]

iters = 5000
valid_choices = valid_choices.view(1, 1, -1, 4)

im = torch.clone(im_original)
im.requires_grad = True

optim = torch.optim.SGD([im], momentum = 0.0, lr = 100)

for it in range(iters):
    
    optim.zero_grad()
    
    im_0 = im[0:W-1, 0:H-1]
    im_1 = im[1:W, 0:H-1]
    im_2 = im[0:W-1, 1:H]
    im_3 = im[1:W, 1:H]
    
    im_tiles = torch.stack([im_0, im_1, im_2, im_3], -1).view(W-1, H-1, 1, 4)
    im_tile_min = im_tiles.min(-1, keepdims=True)[0]
    im_tile_max = im_tiles.max(-1, keepdims=True)[0]
    im_tile_normalize = im_tile_max - im_tile_min
    im_tile_normalize[im_tile_normalize < 1e-5] = 1
    im_tiles = (im_tiles - im_tile_min) / im_tile_normalize

    im_distance_to_valid_choices = ((im_tiles - valid_choices) ** 2).sum(-1).min(2)[0]

    valid_loss = im_distance_to_valid_choices.mean()
    pixel_loss = (im - im_original).abs().mean()
#     pixel_loss = (im - 0).abs().mean()
    
    total_loss = 1 * valid_loss + 1 * pixel_loss
#     total_loss = 0 * valid_loss + 10 * pixel_loss
    total_loss.backward()
    
    optim.step()
    
    if it % (iters // 100) == 0:
        print("it%5d Valid|%8.4f Pixel|%8.4f Total|%8.4f" % (it, valid_loss, pixel_loss, total_loss))
    
# for i in range(2):
#     for sx in range(i, W-1, 2):
#         for sy in range(i, H-1, 2):
#             patch = im[sx:sx+2, sy:sy+2].reshape(4,).astype(np.float64)
#             if patch.max() == patch.min():
#                 continue
#             norm_patch = (patch - patch.min()) / (patch.max() - patch.min())
#             dist = np.array([np.sum(np.abs(norm_patch - vc)) for vc in valid_choices])
#             ix = np.argmin(dist)
#             print(ix, end='')
#             im[sx:sx+2, sy:sy+2] = (valid_choices[ix] * (patch.max() - patch.min()) + patch.min()).reshape(2, 2)
        
print(im)
print(im.max())

torch.Size([64, 64])
it    0 Valid|  0.0146 Pixel|  0.0000 Total|  0.0146
it   50 Valid|  0.1499 Pixel|  1.4747 Total|  1.6247
it  100 Valid|  0.1254 Pixel|  1.2484 Total|  1.3737
it  150 Valid|  0.1206 Pixel|  1.1804 Total|  1.3010
it  200 Valid|  0.1176 Pixel|  1.1468 Total|  1.2644
it  250 Valid|  0.1128 Pixel|  1.1182 Total|  1.2310
it  300 Valid|  0.1112 Pixel|  1.0924 Total|  1.2036
it  350 Valid|  0.1124 Pixel|  1.0706 Total|  1.1831
it  400 Valid|  0.1060 Pixel|  1.0499 Total|  1.1560
it  450 Valid|  0.1125 Pixel|  1.0282 Total|  1.1407
it  500 Valid|  0.1083 Pixel|  1.0074 Total|  1.1157
it  550 Valid|  0.1072 Pixel|  0.9884 Total|  1.0956
it  600 Valid|  0.1171 Pixel|  0.9714 Total|  1.0885
it  650 Valid|  0.1108 Pixel|  0.9543 Total|  1.0651
it  700 Valid|  0.1081 Pixel|  0.9392 Total|  1.0474
it  750 Valid|  0.1112 Pixel|  0.9228 Total|  1.0339
it  800 Valid|  0.1083 Pixel|  0.9065 Total|  1.0148
it  850 Valid|  0.1126 Pixel|  0.8904 Total|  1.0030
it  900 Valid|  0.1089 Pi

In [ ]:
# display(Image.fromarray(im))
concat_images_vertically(
    Image.fromarray(im.cpu().detach().numpy()),
    Image.fromarray(im_original.cpu().detach().numpy()),
    10,
).show()